In [1]:
import os
import json
from dotenv import load_dotenv
import google.genai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from pathlib import Path

# Read the environment variables
load_dotenv()

True

In [2]:
# Set path
CACHE_FILE = "./file_cache.json"
question_pdf_path = "../.example/paper.pdf"
solution_pdf_path = "../.example/solution.pdf"

def load_cache():
    """Read the cache in JSON format"""
    if os.path.exists(CACHE_FILE):
        with open(CACHE_FILE, "r", encoding="utf-8") as f:
            return json.load(f)
    return {}

def save_cache(cache: dict):
    """Save the cache in JSON format"""
    with open(CACHE_FILE, "w", encoding="utf-8") as f:
        json.dump(cache, f, indent=2, ensure_ascii=False)

def upload_if_needed(path: str, cache_key: str):
    """
    Get the file uri if it has already been uploaded.
    If not, upload the file to google and return the uri.
    """

    # Load the cache
    cache = load_cache()

    # If the file uri is already cached, return it.
    if cache_key in cache:
        print(f"✅ 已缓存: {cache_key} → {cache[cache_key]}")
        return cache[cache_key]

    # If not, get the client and upload the file.
    client = genai.Client(api_key=os.environ["GOOGLE_API_KEY"])

    print(f"⏫ 正在上传 {path} ...")
    uploaded = client.files.upload(file=path)
    cache[cache_key] = uploaded.uri
    save_cache(cache)
    print(f"✅ 上传完成: {cache_key} → {uploaded.uri}")
    return uploaded.uri

def delete_all_files():
    client = genai.Client(api_key=os.environ["GOOGLE_API_KEY"])
    files = client.files.list()

    for f in files:
        client.files.delete(name=f.name)
        print(f"🗑️ 已删除: {f.name}")

    # if os.path.exists(CACHE_FILE):
    #     with open(CACHE_FILE, "w", encoding="utf-8") as f:
    #         json.dump({}, f, indent=2, ensure_ascii=False)
    #     print(f"🧹 已清空缓存文件: {CACHE_FILE}")
    # else:
    #     print("ℹ️ 未找到缓存文件，无需清空。")


q_uri = upload_if_needed(question_pdf_path, "paper_pdf_uri")
s_uri = upload_if_needed(solution_pdf_path, "solution_pdf_uri")

⏫ 正在上传 ../.example/paper.pdf ...
✅ 上传完成: paper_pdf_uri → https://generativelanguage.googleapis.com/v1beta/files/o6isoica6d8h
⏫ 正在上传 ../.example/solution.pdf ...
✅ 上传完成: solution_pdf_uri → https://generativelanguage.googleapis.com/v1beta/files/s1b7kk5wrk38


In [3]:
def list_all_uploaded_files():
    client = genai.Client(api_key=os.environ["GOOGLE_API_KEY"])
    files = client.files.list()

    for f in files:
        print(f"""
    URI:   {f.uri}
    NAME:  {f.name}
    STATE: {f.state}
    SIZE:  {f.size_bytes / 1024:.1f} KB
    TIME:  {f.create_time}
    """)

list_all_uploaded_files()



    URI:   https://generativelanguage.googleapis.com/v1beta/files/s1b7kk5wrk38
    NAME:  files/s1b7kk5wrk38
    STATE: FileState.ACTIVE
    SIZE:  9230.1 KB
    TIME:  2025-11-11 14:13:15.100045+00:00
    

    URI:   https://generativelanguage.googleapis.com/v1beta/files/o6isoica6d8h
    NAME:  files/o6isoica6d8h
    STATE: FileState.ACTIVE
    SIZE:  518.9 KB
    TIME:  2025-11-11 14:12:20.298004+00:00
    


In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

agent = create_agent(
    model=llm,
    tools=[],
    system_prompt="You are an Australian high school teacher.",
)

# === 3. 构造消息并执行 ===
question = HumanMessage(content=[
    {"type": "text", "text":
        "You are an Australian high school teacher.\n"
        "Please ONLY summarize the content of the third page of the solution PDF. Do NOT scan the whole PDF. "
    },
    {"type": "media", "mime_type": "application/pdf", "file_uri": q_uri},
    {"type": "media", "mime_type": "application/pdf", "file_uri": s_uri},
])


In [ ]:
for step in agent.stream({"messages": [question]}, stream_mode="values"):
    step["messages"][-1].pretty_print()